# Python and Data Analysis 2 - Cleaning, Preparing, and Organizing Data

**Goal:** The goal of this project is to learn to prepare data for analysis using Pandas.

**Description:** Data often needs to be organized, combined, or cleaned before performing analysis. This project explains a few key ways to accomplish this.

## 2A: Preparing and Cleaning Data

Data *cleaning* refers to the process of ensuring our data is correct and complete. If data is not cleaned, our analysis could produce incorrect results based on corrupt/garbage information. *Preparation* refers to getting our data ready for analysis by putting it into a format we can easily understand.

### Removing Garbage Data

*Garbage data* depends on the type of data we are representing. One common example is if we have a DataFrame with inconsistent types in its columns.

In [6]:
import pandas as pd

df = pd.DataFrame({'x': [1,'more garbage',3,4,5,6,7,8,9,10],
                  'y': [1,4,9,16,25,36,49,64,'garbage',100]})
print(df)

              x        y
0             1        1
1  more garbage        4
2             3        9
3             4       16
4             5       25
5             6       36
6             7       49
7             8       64
8             9  garbage
9            10      100


Clearly, we want `x` and `y` to contain numerical values. Therefore rows 1 and 8 contain garbage data. We can try converting them to numbers to fix this.

In [193]:
df = df.apply(pd.to_numeric, errors='coerce') # Go through the DataFrame and force every value to be a number
print(df.dtypes)
print(df)

x    float64
y    float64
dtype: object
      x      y
0   1.0    1.0
1   NaN    4.0
2   3.0    9.0
3   4.0   16.0
4   5.0   25.0
5   6.0   36.0
6   7.0   49.0
7   8.0   64.0
8   9.0    NaN
9  10.0  100.0


Notice the strings have been replaced `NaN`. This is a special value considered to be a `float`, so it is consistent with the other numerical values in its respective column. However, when NaN is involved in a calculation, it results in NaN output. Therefore, we typically want to remove these values.

In [194]:
# NaN + 4 = NaN
print(df.iloc[1,0] + df.iloc[1,1]) # Add the x value from row 1 (NaN) to the y value from row 1 (4)

nan


We can remove `NaN` values in two ways. If we want to completely delete the row containing the NaN, we can use `dropna()`. If we want to replace the NaN with some constant value, we can use `fillna(replacement_value)`. The first example removes rows with NaN, and the second example replaces the NaNs with the value 0. 

In [195]:
df1 = df.dropna()
# print("Dropping NaNs")
# print(df1)

df2 = df.fillna(0)
print("\nFilling NaNs")
print(df2)


Filling NaNs
      x      y
0   1.0    1.0
1   0.0    4.0
2   3.0    9.0
3   4.0   16.0
4   5.0   25.0
5   6.0   36.0
6   7.0   49.0
7   8.0   64.0
8   9.0    0.0
9  10.0  100.0


### Preparing Data

#### Dropping Columns
When getting data ready for analysis, we might want to select subsections of our data and crop out the rest. For example, let's say we are interested in the `date`, `close` and `volume` for Microsoft stock. Using the techniques discussed earlier, we can do the following.

In [196]:
df = pd.read_csv('MSFT.csv')
df = df[['date', 'close', 'volume']]
print(df.head())

         date   close      volume
0  1986-03-13  0.0972  1031788800
1  1986-03-14  0.1007   308160000
2  1986-03-17  0.1024   133171200
3  1986-03-18  0.0998    67766400
4  1986-03-19  0.0981    47894400


Alternatively, if there were some columns which we were just not interested in, we could `drop` those and leave the rest. Note: we need `axis=1` to tell Pandas to drop columns and not rows.

In [197]:
df1 = pd.read_csv('MSFT.csv')
df1 = df1.drop(['close'], axis=1)
print(df1.head())

         date    open    high     low      volume
0  1986-03-13  0.0885  0.1016  0.0885  1031788800
1  1986-03-14  0.0972  0.1024  0.0972   308160000
2  1986-03-17  0.1007  0.1033  0.1007   133171200
3  1986-03-18  0.1024  0.1033  0.0990    67766400
4  1986-03-19  0.0998  0.1007  0.0972    47894400


#### Changing Data Types
After collecting the data we want, we sometimes need to change the type of a column. A common example is converting `date` from `string` to `datetime`.

In [198]:
df['date'] = pd.to_datetime(df['date']) # Convert 'date' column from string to datetime
print(df['date'].head()) # Display the first few rows of the 'date' column

0   1986-03-13
1   1986-03-14
2   1986-03-17
3   1986-03-18
4   1986-03-19
Name: date, dtype: datetime64[ns]


#### Renaming Columns
We can easily rename our columns with `rename`. We supply a dictionary, mapping the old names to new ones.

In [199]:
df = df.rename(columns={'date': 'Date', 'close': 'Closing Price', 'volume': 'Trading Volume'})
print(df.head())

        Date  Closing Price  Trading Volume
0 1986-03-13         0.0972      1031788800
1 1986-03-14         0.1007       308160000
2 1986-03-17         0.1024       133171200
3 1986-03-18         0.0998        67766400
4 1986-03-19         0.0981        47894400


#### Changing Index
Finally, we might want to relabel our index, by setting its values to be the `Date` column. This is useful if we frequently want to access rows by `Date`.

In [200]:
df = df.set_index('Date')
print(df.head())


            Closing Price  Trading Volume
Date                                     
1986-03-13         0.0972      1031788800
1986-03-14         0.1007       308160000
1986-03-17         0.1024       133171200
1986-03-18         0.0998        67766400
1986-03-19         0.0981        47894400


In [201]:
print(df.loc["1986-03-17"])
print(df.loc["1986-03-13"])


Closing Price     1.024000e-01
Trading Volume    1.331712e+08
Name: 1986-03-17 00:00:00, dtype: float64
Closing Price     9.720000e-02
Trading Volume    1.031789e+09
Name: 1986-03-13 00:00:00, dtype: float64


Note, the new index is the old `Date` column, which has been removed. We can no longer access `Date` using `df['Date']` because it is no longer considered a column. We would instead access the index using `df.index`, `df.index.values`, or `df.index.values.tolist()`.

**Challenge**: Reload the MSFT CSV into a variable called `df1`, then drop the open and volume columns from it, convert the date column to datetime, then use the date column as the index. **Important**: don't name the new dataframe `df`.

In [202]:
df3 = pd.read_csv('MSFT.csv')
df3 = df3.drop(['open', 'volume'], axis=1)
# print(df3.head())

df3['date'] = pd.to_datetime(df3['date'])
# print(df3.head())
df3 = df3.set_index('date')
print(df3.head())

print(df3.loc['1986-03-17'])

# df1 = pd.read_csv('MSFT.csv')
# df1 = df1.drop(['open', 'volume'], axis=1)
# df1['date'] = pd.to_datetime(df1['date'])
# df1 = df1.set_index('date')
# print(df1.head())

              high     low   close
date                              
1986-03-13  0.1016  0.0885  0.0972
1986-03-14  0.1024  0.0972  0.1007
1986-03-17  0.1033  0.1007  0.1024
1986-03-18  0.1033  0.0990  0.0998
1986-03-19  0.1007  0.0972  0.0981
high     0.1033
low      0.1007
close    0.1024
Name: 1986-03-17 00:00:00, dtype: float64


### Sorting Data

Finally, we might want to sort our data by one or more columns. Lets say we want to sort the data in increasing order by `volume`.

We use the syntax `df-name.sort_values()` which takes the parameters:
 - `by=`: The column name or column list to sort by
 - `ascending=`: Whether you want to sort in increasing (True) or decreasing (False) order

In [203]:
msft = pd.read_csv('MSFT.csv')
msft = msft.sort_values(by=['volume', 'close'], ascending=True)
print(msft)

            date     open     high      low    close      volume
62    1986-06-11   0.1137   0.1137   0.1120   0.1137     2304000
137   1986-09-26   0.1007   0.1016   0.0998   0.1007     2332800
114   1986-08-25   0.1024   0.1024   0.1007   0.1024     2419200
132   1986-09-19   0.1033   0.1042   0.1024   0.1033     2505600
100   1986-08-05   0.0981   0.0990   0.0972   0.0972     2822400
...          ...      ...      ...      ...      ...         ...
851   1989-07-25   0.3819   0.3958   0.3576   0.3611   326028816
5078  2006-04-28  24.2600  24.5000  24.0000  24.1500   591052200
2081  1994-06-06   3.2894   3.4144   3.2344   3.4063   764504000
754   1989-03-07   0.4149   0.4167   0.3698   0.3715   788688000
0     1986-03-13   0.0885   0.1016   0.0885   0.0972  1031788800

[8634 rows x 6 columns]


If we wanted to sort by multiple columns, we pass a list into the `by=` parameter. For example, to sort by `volume` and then `close`, we would do `msft.sort_values(by=['volume', 'close'], ascending=True)`.

## 2B: Combining DataFrames

Sometimes, data is scattered across multiple DataFrames. In that case, we need to be able to join DataFrames together. There are several ways to do this in Pandas, but this tutorial will cover `merge`. Suppose we have the following two DataFrames.

In [204]:
classes = pd.DataFrame({
    'class': [1,2,3,4,5],
    'teacher': ['Bill Ryan', 'Sue Kim', 'Arun Gupta', 'Darnell Williams', 'Emily Coles'],
})

teachers = pd.DataFrame({
    'teacher': ['Sue Kim', 'Arun Gupta', 'Emily Coles', 'Darnell Williams', 'Ben Smith', 'Nicole Ang'],
    'age': [43, 56, 32, 44, 32, 53],
    'experience': [4, 5, 2, 6, 1, 4],
})

print("Classes")
print(classes)
print("\nTeachers")
print(teachers)

Classes
   class           teacher
0      1         Bill Ryan
1      2           Sue Kim
2      3        Arun Gupta
3      4  Darnell Williams
4      5       Emily Coles

Teachers
            teacher  age  experience
0           Sue Kim   43           4
1        Arun Gupta   56           5
2       Emily Coles   32           2
3  Darnell Williams   44           6
4         Ben Smith   32           1
5        Nicole Ang   53           4


The `classes` DataFrame contains information about 5 classes and the teachers that teach them. The `teachers` DataFrame contains information about each teacher: their age, and years of experience. We want to obtain one DataFrame, with information about class, teacher, age, and experience. There are four ways to do this. 

When joining DataFrames, we visualize the operation as adding the *right* DataFrame to the *left* one. We join based on a column that contains the same data between the two DataFrames. In this case, the shared piece of data is the teacher.

The syntax is `left_df.merge(right_df, on='', how='')`.
 - `on=''`: Replace the quotes with the common column between the two DataFrames
 - `how=''`: Replace the quotes with the type of join: `'left'`, `'right'`, `'outer'`, `'inner'`.

### Left Join

A *left join* keeps all the rows in the left DataFrame, and adds entries from the right DataFrame where the `on` column matches.

In [205]:
combined = classes.merge(teachers, on='teacher', how='left')
print(combined)

   class           teacher   age  experience
0      1         Bill Ryan   NaN         NaN
1      2           Sue Kim  43.0         4.0
2      3        Arun Gupta  56.0         5.0
3      4  Darnell Williams  44.0         6.0
4      5       Emily Coles  32.0         2.0


The ages and experience for Sue, Arun, Darnell, and Emily have been added to the left DataFrame. Bill Ryan remains because he was in the left DataFrame, but does not have an age or experience because he is not present in the right DataFrame.

### Right Join

A *right join* keeps all the rows in the right DataFrame, and adds entries from the left DataFrame where the `on` column matches.

In [206]:
combined = classes.merge(teachers, on='teacher', how='right')
print(combined)

   class           teacher  age  experience
0    2.0           Sue Kim   43           4
1    3.0        Arun Gupta   56           5
2    5.0       Emily Coles   32           2
3    4.0  Darnell Williams   44           6
4    NaN         Ben Smith   32           1
5    NaN        Nicole Ang   53           4


As before, the ages and experience for Sue, Arun, Darnell, and Emily have been added. Bill Ryan was not in the right DataFrame, so his name is not present, and his class has disappeared as well. Ben Smith and Nicole Ang were present in the right DataFrame, so they are included, but they do not have an associated class.

### Inner Join

An *inner join* keeps all the rows where the `on` column matches - in other words, where the `teacher` is present in both DataFrames.

In [207]:
combined = classes.merge(teachers, on='teacher', how='inner')
print(combined)

   class           teacher  age  experience
0      2           Sue Kim   43           4
1      3        Arun Gupta   56           5
2      4  Darnell Williams   44           6
3      5       Emily Coles   32           2


The ages and experience for Sue, Arun, Darnell, and Emily are included because they are present in both DataFrames. Bill Ryan is only present in the left DataFrame, so he is not included. Ben Smith and Nicole Ang are only present in the right DataFrame, so they are not included.

### Outer Join

An *outer join* keeps all entries from both DataFrames. Where values don't exist, it fills the cell with `NaN`.

In [208]:
combined = classes.merge(teachers, on='teacher', how='outer')
print(combined)

   class           teacher   age  experience
0    3.0        Arun Gupta  56.0         5.0
1    NaN         Ben Smith  32.0         1.0
2    1.0         Bill Ryan   NaN         NaN
3    4.0  Darnell Williams  44.0         6.0
4    5.0       Emily Coles  32.0         2.0
5    NaN        Nicole Ang  53.0         4.0
6    2.0           Sue Kim  43.0         4.0


Every teacher is included in this combination. If they don't have an age or experience (Bill Ryan) it is filled with NaN. If they don't have a class (Ben Smith, Nicole Ang), it is filled with NaN. Try and think about which types of joins would be useful in different scenarios.

## 2C: Grouping Data

Finally, if we have a large DataFrame, we might be interested organizing the data into groups. We can do this easily using `groupby`. In the following example, we create dataframes for Microsoft, Apple, Amazon, and Google stock prices. Then we `append` them together into one large DataFrame. 

In [27]:
stock_names = ['MSFT', 'AAPL', 'AMZN', 'GOOG']

df = pd.DataFrame()

df_list = []  # create a list for saving DataFrame

for stock_name in stock_names:
    stock_df = pd.read_csv(f'{stock_name}.csv')  # read CSV
    stock_df['name'] = stock_name  # add column with company's name
    df_list.append(stock_df)  # add to the list

df = pd.concat(df_list, ignore_index=True) 
# ignore_index=True for updating indices
print(df)

             date       open       high        low      close      volume  \
0      1986-03-13     0.0885     0.1016     0.0885     0.0972  1031788800   
1      1986-03-14     0.0972     0.1024     0.0972     0.1007   308160000   
2      1986-03-17     0.1007     0.1033     0.1007     0.1024   133171200   
3      1986-03-18     0.1024     0.1033     0.0990     0.0998    67766400   
4      1986-03-19     0.0998     0.1007     0.0972     0.0981    47894400   
...           ...        ...        ...        ...        ...         ...   
25960  2020-06-08  1422.3400  1447.9900  1422.3400  1446.6100     1401827   
25961  2020-06-09  1445.3600  1468.0000  1443.2100  1456.1600     1409249   
25962  2020-06-10  1459.5400  1474.2600  1456.2700  1465.8500     1521378   
25963  2020-06-11  1442.4800  1454.4700  1402.0000  1403.8400     1983710   
25964  2020-06-12  1428.4900  1437.0000  1386.0200  1413.1800     1946367   

       name  
0      MSFT  
1      MSFT  
2      MSFT  
3      MSFT  
4    

Let's say `df` was the only DataFrame we had. Without knowing where each stock's price data begins and ends, there is no way to separate data by stock (the `name` column). Instead, we use `groupby` to create an object that stores the grouped DataFrames. Here, we want to group the rows in the DataFrame by `name`.

In [ ]:
groups = df.groupby('name') # Creates a grouping of the DataFrame by column 'name'
print(groups)

We cannot display `groups` directly, because it is not a DataFrame. Instead, we can access specific groups stored in `groups`. The following code gets all the rows in the group `'MSFT'`, returns them as a DataFrame, and stores them in the variable `msft_data`. The resulting DataFrame contains price information for only Microsoft's stock.

In [22]:
msft_data = groups.get_group('AAPL')
print(msft_data)

             date      open      high       low     close     volume  name
8634   1980-12-12    0.5134    0.5155    0.5134    0.5134  117258400  AAPL
8635   1980-12-15    0.4889    0.4889    0.4866    0.4866   43971200  AAPL
8636   1980-12-16    0.4530    0.4530    0.4509    0.4509   26432000  AAPL
8637   1980-12-17    0.4620    0.4643    0.4620    0.4620   21610400  AAPL
8638   1980-12-18    0.4755    0.4777    0.4755    0.4755   18362400  AAPL
...           ...       ...       ...       ...       ...        ...   ...
18588  2020-06-08  330.2500  333.6000  327.3200  333.4600   23851975  AAPL
18589  2020-06-09  332.1400  345.6100  332.0100  343.9900   36928091  AAPL
18590  2020-06-10  347.9000  354.7700  346.0900  352.8400   41355786  AAPL
18591  2020-06-11  349.3100  351.0600  335.4800  335.9000   49567675  AAPL
18592  2020-06-12  344.7200  347.8000  334.2233  338.8000   50036513  AAPL

[9959 rows x 7 columns]


To practice, try getting the data for Apple, Google, and Amazon from `groups`. We don't have to group by `name`, but we can also group by other columns as well. For example, if we wanted to compare data across all stocks on a given date, we could `groupby('date')`.

In [28]:
dates_groups = df.groupby('date') # A grouping of the DataFrame by date

To view the stock prices on a given day (for example January 3, 2018), we can `get_group` for the day we are interested in.

In [29]:
target_data = dates_groups.get_group('2018-01-03')
print(target_data)

             date      open     high      low    close    volume  name
8019   2018-01-03    86.055    86.51    85.97    86.35  26061439  MSFT
17978  2018-01-03   172.530   174.55   171.96   172.23  29517899  AAPL
23785  2018-01-03  1188.300  1205.49  1188.30  1204.20   3108793  AMZN
25350  2018-01-03  1064.310  1086.29  1063.21  1082.48   1430170  GOOG


**Challenge**: Find the most common volume amount across all four stocks using `groupby`, then print the rows in the dataframe with that volume using `get_group`.

In [ ]:
t = df.groupby('volume')
# print(t.get_group(7900))
# print(t.groups.items())
# Sort based on the number of values in each group
most_common = max(t.groups.items(), key=lambda item: len(item[1]))
# print(most_common)
# print(t.get_group(most_common[0]))

             date    open    high     low   close  volume  name
24403  2014-04-01  558.71  568.45  558.71  567.16    7900  GOOG
dict_items([(7900, Index([24403], dtype='int64')), (10800, Index([24402], dtype='int64')), (13100, Index([24400], dtype='int64')), (41100, Index([24401], dtype='int64')), (146700, Index([24404], dtype='int64')), (250376, Index([9844], dtype='int64')), (347200, Index([9158], dtype='int64')), (347518, Index([25847], dtype='int64')), (487200, Index([18748], dtype='int64')), (527223, Index([24841], dtype='int64')), (536996, Index([25324], dtype='int64')), (571200, Index([18653], dtype='int64')), (574800, Index([18637], dtype='int64')), (586981, Index([25830], dtype='int64')), (587421, Index([25073], dtype='int64')), (590400, Index([18654], dtype='int64')), (591600, Index([18603], dtype='int64')), (599200, Index([11570], dtype='int64')), (620400, Index([18641], dtype='int64')), (623944, Index([25093], dtype='int64')), (624000, Index([18660], dtype='int64')), (66775